Iterables and Iterators

Iterator vs Iterable vs Generator

An iterable is an object that can return an iterator., Any object with state that has an __iter__ method and returns an iterator is an iterable. 

In [2]:
#Iterable classes:
# Iterable classes define an __iter__ and a __next__ method.
class MyIterable:
    def __iter__(self):
        return self
    def __next__(self):
        #code
        pass

Extract values one by one

In [4]:
s = {1, 2} # or list or generator or even iterator 
i = iter(s) # get iterator
a = next(i) # a = 1
print(a)

1


In [5]:
b = next(i)
b

2

In [6]:
c = next(i)

StopIteration: 

Iterating over entire iterable

In [7]:
s = {1, 2, 3}
# get every element in s
for a in s:
    print(a)

1
2
3


In [8]:
# copy into list 
l1 = list(s)
print(l1)

[1, 2, 3]


In [12]:
# use list comprehension
l2 = [a*2 for a in s if a>2] # 12 = [6]

Verify only one element in iterable

In [20]:
a, = iterable
def foo():
    yield 1
a, = foo() # a = 1
nums = [1, 2, 3]
a, = nums   #valueError: too many values to unpack

NameError: name 'iterable' is not defined

What can be iterable

# Generators return iterables:
def foo(): # foo isn't iterable yet 
    yeild 1
res = foo()  #...but res already is 

Iterator isn't reentrant!

In [24]:
def gen():
    yield 1
iterable = gen()
for a in iterable:
    print(a)
# What was the first item of iterable? No way to get it now.
# Only to get a new iterator
gen()

1


<generator object gen at 0x0000024483C96B20>

Partial functions

Raise the power

In [25]:
def raise_power(x, y):
    return x**y

In [31]:
# if y be one of [3, 4, 5]
def rais(x, y):
    if y in (3, 4, 5):
        return x**Y
    raise NumberNotInRangeException("You should provide a valid exponent")

# implementing cases partially
from functors import partial
raise_to_three = parital(raise, y=3)
raise_to_four = parital(raise, y=4)
raise_to_five = parital(raise, y=5)

Decorators

In [33]:
# Decorator function
# This simplest decorator does nothing to the function being decorated. Such
# minimal decorators can occasionally be used as a kind of code markers.
def super_secret_function(f):
    return f
@super_secret_function
def my_function():
    print("This is my secret function")

In [34]:
def disabled(f):
    """
    This function returns nothing, and hence removes the decorated function
    from the local scope.
    """
    pass
@disabled
def my_function():
    print("This function can no longer be called...")
my_function()

TypeError: 'NoneType' object is not callable

Thus, we usually define a new function inside the decorator and return it. This new function would first do
something that it needs to do, then call the original function, and finally process the return value.

In [37]:
# This is the decorator
def print_args(func):
    def inner_func(*args, **kwargs):
        print(args)
        print(kwargs)
        return func(*args, **kwargs) # call the original function with its arguments.
    return inner_func
@print_args
def multiply(num_a, num_b):
    return num_a * num_b
print(multiply(3, 5))
# 'args' that the function receives is returned.
# 'kwargs', empty because we didn't specify keyword arguments.
# The result of the function is returned

(3, 5)
{}
15


Decorator class

In [38]:
# Example
class Decorator(object):
    """Simple decorator class."""
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print('Befor the function call.')
        res = self.func(*args, **kwargs)
        print('After the function call.')
        return res
@Decorator
def testfunc():
    print('Inside the function.')
testfunc()

Befor the function call.
Inside the function.
After the function call.


Note that a function decorated with a class decorator will no longer be considered a "function" from type-checking
perspective:

In [39]:
# example
import types
isinstance(testfunc, types.FunctionType)

False

In [40]:
type(testfunc)

__main__.Decorator

Decorating Methods

In [41]:
from types import MethodType
class Decorator(object):
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print('Inside the decorator.')
        return self.func(*args, **kwargs)
    def __get__(self, instance, cls):
        # Return a Method if it is called on an instance
        return self if instance is None else MethodType(self, instance)
class Test(object):
    @Decorator
    def __init__(self):
        pass
a = Test()

Inside the decorator.


Warning!
Class Decorators only produce one instance for a specific function so decorating a method with a class decorator
will share the same decorator between all instances of that class:

In [42]:
from types import MethodType
class CountCallsDecorator(object):
    def __init__(self, func):
        self.func = func
        self.ncalls = 0 # Number of calls of this method
    def __call__(self, *args, **kwargs):
        self.ncalls += 1 # Increment the calls counter
        return self.func(*args, **kwargs)
    def __get__(self, instance, cls):
        return self if instance is None else MethodType(self, instance)
class Test(object):
    def __init__(self):
        pass
    @CountCallsDecorator
    def do_something(self):
        return 'something was done'
a = Test()
a.do_something()

'something was done'

In [43]:
a.do_something.ncalls

1

In [44]:
b = Test()
b.do_something()

'something was done'

In [45]:
b.do_something.ncalls

2

Decorator with arguments (decorator factory):
A decorator takes just one argument: the function to be decorated. There is no way to pass other arguments.
But additional arguments are often desired. The trick is then to make a function which takes arbitrary arguments
GoalKicker.com – Python® Notes for Professionals 206
and returns a decorator

In [50]:
@decoratorfactory # without parentheses
def test():
    pass
test()

TypeError: 'NoneType' object is not callable

In [51]:
# Decorator classes
def decoratorfactory(*decorator_args, **decorator_kwargs):
    class Decorator(object):
        def __init__(self, func):
            self.func = func
        def __call__(self, *args, **kwargs):
            print('Inside the decorator with arguments {}'.format(decorator_args))
            return self.func(*args, **kwargs)
    return Decorator
@decoratorfactory(10)
def test():
    pass
test()

Inside the decorator with arguments (10,)


Making a decorator look like the decorated function

In [52]:
from functools import wraps
def decorator(func):
    # Copies the docstring, name, annotations and module to the decorator
    @wraps(func)
    def wrapped_func(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapped_func
@decorator
def test():
    pass
test.__name__

'test'

In [54]:
# As a class
class Decorator(object):
    def __init__(self, func):
        # copies name, module, annotations and docstring to the instance.
        self._wrapped = wraps(func)(self)
    def __call__(self, *args, **kwargs):
        return self._wrapped(*args, **kwargs)
@Decorator
def test():
    """Docstring of test."""
    pass
test.__doc__

'Docstring of test.'